In [1]:
import numpy as np
import ray
import ray.rllib

import q1physrl.env

/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
ray.init()

2020-01-30 23:16:00,485	WARNING worker.py:682 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2020-01-30 23:16:00,488	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-30 23:16:00,573	INFO resource_spec.py:212 -- Starting Ray with 1.9 GiB memory available for workers and up to 0.97 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-30 23:16:00,854	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.
2020-01-30 23:16:00,927	WARNING services.py:1004 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '192.168.178.24',
 'redis_address': '192.168.178.24:43746',
 'object_store_address': '/tmp/ray/session_2020-01-30_23-16-00_488058_13890/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-30_23-16-00_488058_13890/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-01-30_23-16-00_488058_13890'}

In [3]:
trainer = ray.rllib.agents.ppo.PPOTrainer(
# trainer = ray.rllib.agents.a3c.A2CTrainer(
    env=q1physrl.env.PhysEnv,
    config={"env_config": {"num_envs": 100}, "gamma": 0.99}
)

2020-01-30 23:16:05,731	INFO trainer.py:377 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-01-30 23:16:05,739	ERROR logger.py:328 -- pip install 'ray[tune]' to see TensorBoard files.
2020-01-30 23:16:05,741	WARNING logger.py:417 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2020-01-30 23:16:05,744	INFO trainer.py:524 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=13932) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=13932)   warnings.warn(msg)
(pid=13934) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=13934)   warnings.warn(msg)
(pid=13932) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=13932)   warnings.warn(msg)
(pid=13934) /home/matt/.virtualenvs/100m-rl/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma modu

2020-01-30 23:16:12,967	WARNING util.py:41 -- Install gputil for GPU system monitoring.
2020-01-30 23:16:12,968	WARNING util.py:44 -- Install psutil to monitor system performance.


In [ ]:
i = 0
while True:
    print(trainer.train()['episode_reward_mean'])
    if i % 20 == 0:
        print(trainer.save())
    i += 1

(pid=13932) /home/matt/100m-rl/q1physrl/phys.py:99: RuntimeWarning: invalid value encountered in true_divide
(pid=13932)   wish_vel / unclipped_wish_speed[:, None],
(pid=13932) /home/matt/100m-rl/q1physrl/phys.py:104: RuntimeWarning: invalid value encountered in true_divide
(pid=13932)   (wish_speed / unclipped_wish_speed),
(pid=13932) /home/matt/100m-rl/q1physrl/phys.py:89: RuntimeWarning: invalid value encountered in true_divide
(pid=13932)   return np.where((speed > 0)[:, None], h_vel * (new_speed / speed)[:, None], h_vel)
(pid=13934) /home/matt/100m-rl/q1physrl/phys.py:99: RuntimeWarning: invalid value encountered in true_divide
(pid=13934)   wish_vel / unclipped_wish_speed[:, None],
(pid=13934) /home/matt/100m-rl/q1physrl/phys.py:104: RuntimeWarning: invalid value encountered in true_divide
(pid=13934)   (wish_speed / unclipped_wish_speed),
(pid=13934) /home/matt/100m-rl/q1physrl/phys.py:89: RuntimeWarning: invalid value encountered in true_divide
(pid=13934)   return np.where((sp

In [ ]:
trainer.save()

In [ ]:
# trainer.restore('/home/matt/ray_results/A2C_PhysEnv_2020-01-27_21-54-210mybah6w/checkpoint_38/checkpoint-38')

In [ ]:
env = q1physrl.env.PhysEnv({'num_envs': 1})
obs = env.vector_reset()
done = None

action_list = []
obs_list = []
rewards_list = []

while done is None or not done[0]:
    obs_list.append(obs)
    action = trainer.compute_action(obs[0])
    action_list.append(action)
    obs, reward, done, _ = env.vector_step(action[None])
    rewards_list.append(reward)
    
actions = np.array(action_list)
obss = np.array(obs_list)
rewards = np.array(rewards_list)

In [ ]:
import pandas as pd
pd.DataFrame({'observed yaw': obss[:, 0, 0], 'yaw action': actions[:, 0]})

In [ ]:
obss[0]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(obss[:50, 0, 0])

In [ ]:
plt.plot((actions[:, 0] - 1).cumsum() * 0.014 * 360)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(actions[:36, 0])

In [ ]:
actions[:, 0].mean()